In [1]:
from sympy import *

In [7]:
I = Matrix(3,3, lambda i,j: 1 if i==j else 0)
def _Matrix(M,ij):
    return Matrix(3,3,[Symbol('{}[{}]'.format(M,i)) for i in ij])
def SymMatrix(M):
    return _Matrix(M, (0,3,5,3,1,4,5,4,2))
def UnsymMatrix(M):
    return _Matrix(M, (0,1,2,3,4,5,6,7,8))

In [14]:
print '    @classmethod'
print '    def param_names(cls, n):'
N = 5
ij = []
for n in range(N+2):
    ij.extend([(i,j) for i in range(n)[::-1] for j in range(n) if i+j==n-1])
ij = ij[1:]
params = '        return [{}]'.format(','.join(["'C{}{}'".format(i,j) for i,j in ij]))
print params

    @classmethod
    def param_names(cls, n):
        return ['C10','C01','C20','C11','C02','C30','C21','C12','C03','C40','C31','C22','C13','C04','C50','C41','C32','C23','C14','C05']


In [28]:
F = UnsymMatrix('F')
FtF = F.T * F
print 'def symsq(F):'
ij = {0:(0,0),1:(1,1),2:(2,2),3:(0,1),4:(1,2),5:(0,2)}
print '    X = zeros(6)'
for I in range(6):
    i,j = ij[I]
    print '    X[{}] = {}'.format(I, FtF[i,j])
print '    return X'

def symsq(F):
    X = zeros(6)
    X[0] = F[0]**2 + F[3]**2 + F[6]**2
    X[1] = F[1]**2 + F[4]**2 + F[7]**2
    X[2] = F[2]**2 + F[5]**2 + F[8]**2
    X[3] = F[0]*F[1] + F[3]*F[4] + F[6]*F[7]
    X[4] = F[1]*F[2] + F[4]*F[5] + F[7]*F[8]
    X[5] = F[0]*F[2] + F[3]*F[5] + F[6]*F[8]
    return X


In [16]:
F = UnsymMatrix('F')
A = SymMatrix('A')
dA = A.det()
dF = F.det()
print 'def det(A):'
print '    if issym(A):'
print '        X = {}'.format(dA)
print '    else:'
print '        X = {}'.format(dF).replace('F', 'A')
print '    return X'

def det(A):
    if issym(A):
        X = A[0]*A[1]*A[2] - A[0]*A[4]**2 - A[1]*A[5]**2 - A[2]*A[3]**2 + 2*A[3]*A[4]*A[5]
    else:
        X = A[0]*A[4]*A[8] - A[0]*A[5]*A[7] - A[1]*A[3]*A[8] + A[1]*A[5]*A[6] + A[2]*A[3]*A[7] - A[2]*A[4]*A[6]
    return X


In [18]:
A = SymMatrix('A')
print 'def invariants(A, itype=0):'
I1 = A.trace()
I2 = (I1 ** 2 - (A * A).trace()) / 2
print '    I1 = {}'.format(I1)
print '    I2 = {}'.format(I2)
print '    I3 = det(A)'
print '    return I1, I2, I3'

def invariants(A, itype=0):
    I1 = A[0] + A[1] + A[2]
    I2 = -A[0]**2/2 - A[1]**2/2 - A[2]**2/2 - A[3]**2 - A[4]**2 - A[5]**2 + (A[0] + A[1] + A[2])**2/2
    I3 = det(A)
    return I1, I2, I3


In [24]:

NCI = (1,0,2,1,0,3,2,1,0)
NCJ = (0,1,0,1,2,0,1,2,3)

u = ['u = zeros(2)']
for k in range(9):
    i = NCI[k]
    j = NCJ[k]
    w = 'c[{0}] * ((IB1 - 3.) ** {1}) * ((IB2 - 3.) ** {2})'.format(k, i, j)
    u.append('u[1] += {0}'.format(w))
for k in range(3):
    K = k + 1
    w = '1. / c[{0}] * (Jac - 1.) ** {1}'.format(9+k, 2*K)
    u.append('if (c[{0}] > 0.) u[0] += {1}'.format(9+k, w))
u.append('u[0] = u[0] + u[1]')

du = ['du = zeros(3)']
for k in range(9):
    i = NCI[k]
    j = NCJ[k]
    if i > 0:
        fac = i
        w1 = '(IB1 - 3.) ** {0}'.format(i-1)
        w2 = '(IB2 - 3.) ** {0}'.format(j)
        w = '{0:.1f} * c[{1}] * ({2}) * ({3})'.format(fac, k, w1, w2)
        du.append('du[0] += {0}'.format(w))

    if j > 0:
        fac = j
        w1 = '(IB1 - 3.) ** {0}'.format(i)
        w2 = '(IB2 - 3.) ** {0}'.format(j-1)
        w = '{0:.1f} * c[{1}] * ({2}) * ({3})'.format(fac, k, w1, w2)
        du.append('du[1] += {0}'.format(w))

for k in range(3):
    K = k + 1
    fac = 2 * K
    if fac <= 0:
        continue
    w = '{0:.1f} / c[{1}] * (Jac - 1.) ** {2}'.format(fac, 9+k, 2*K-1)
    du.append('if (c[{0}] > 0.) du[2] += {1}'.format(9+k, w))

d2u = ['ddu = zeros(6)']
for k in range(9):
    i = NCI[k]
    j = NCJ[k]

    fac = i * (i - 1)
    if fac > 0:
        w1 = '(IB1 - 3.) ** {0}'.format(i-2)
        w2 = '(IB2 - 3.) ** {0}'.format(j)
        w = '{0:.1f} * c[{1}] * ({2}) * ({3})'.format(fac, k, w1, w2)
        d2u.append('ddu[0] += {0}'.format(w))

    fac = j * (j - 1)
    if fac > 0:
        w1 = '(IB1 - 3.) ** {0}'.format(i)
        w2 = '(IB2 - 3.) ** {0}'.format(j-2)
        w = '{0:.1f} * c[{1}] * ({2}) * ({3})'.format(fac, k, w1, w2)
        d2u.append('ddu[1] += {0}'.format(w))

    fac = i * j
    if fac > 0:
        w1 = '(IB1 - 3.) ** {0}'.format(i-1)
        w2 = '(IB2 - 3.) ** {0}'.format(j-1)
        w = '{0:.1f} * c[{1}] * ({2}) * ({3})'.format(fac, k, w1, w2)
        d2u.append('ddu[3] += {0}'.format(w))

for k in range(3):
    K = k + 1
    fac = 2 * K * (2 * K - 1)
    if fac <= 0:
        continue
    w = '{0:.1f} / c[{1}] * (Jac - 1.) ** {2}'.format(fac, 9+k, 2*K-2)
    d2u.append('if (c[{0}] > 0.) ddu[2] += {1}'.format(9+k, w))

d3u = ['dddu = zeros(6)']
for k in range(3):
    K = k + 1
    fac = 4 * K * (2 * K - 1) * (K - 1)
    if fac <= 0:
        continue
    w = '{0:.1f} / c[{1}] * (Jac - 1.) ** {2}'.format(fac, 9+k, 2*K-3)
    d3u.append('if (c[{0}] > 0.) dddu[5] += {1}'.format(9+k, w))

print '''
def uhyper(c, IB1, IB2, Jac):
    """HYPERELASTIC POLYNOMIAL MODEL"""'''

s = 'Energies'
print '    # {0}'.format(s)
print '    {0}'.format('\n    '.join(u))

s = 'First Derivatives'
print '\n    # {0}'.format(s)
print '    {0}'.format('\n    '.join(du))

s = 'Second Derivatives'
print '\n    # {0}'.format(s)
print '    {0}'.format('\n    '.join(d2u))

s = 'Third Derivatives'
print '\n    # {0}'.format(s)
print '    {0}'.format('\n    '.join(d3u))

print '    return du, ddu, dddu'


def uhyper(c, IB1, IB2, Jac):
    """HYPERELASTIC POLYNOMIAL MODEL"""
    # Energies
    u = zeros(2)
    u[1] += c[0] * ((IB1 - 3.) ** 1) * ((IB2 - 3.) ** 0)
    u[1] += c[1] * ((IB1 - 3.) ** 0) * ((IB2 - 3.) ** 1)
    u[1] += c[2] * ((IB1 - 3.) ** 2) * ((IB2 - 3.) ** 0)
    u[1] += c[3] * ((IB1 - 3.) ** 1) * ((IB2 - 3.) ** 1)
    u[1] += c[4] * ((IB1 - 3.) ** 0) * ((IB2 - 3.) ** 2)
    u[1] += c[5] * ((IB1 - 3.) ** 3) * ((IB2 - 3.) ** 0)
    u[1] += c[6] * ((IB1 - 3.) ** 2) * ((IB2 - 3.) ** 1)
    u[1] += c[7] * ((IB1 - 3.) ** 1) * ((IB2 - 3.) ** 2)
    u[1] += c[8] * ((IB1 - 3.) ** 0) * ((IB2 - 3.) ** 3)
    if (c[9] > 0.) u[0] += 1. / c[9] * (Jac - 1.) ** 2
    if (c[10] > 0.) u[0] += 1. / c[10] * (Jac - 1.) ** 4
    if (c[11] > 0.) u[0] += 1. / c[11] * (Jac - 1.) ** 6
    u[0] = u[0] + u[1]

    # First Derivatives
    du = zeros(3)
    du[0] += 1.0 * c[0] * ((IB1 - 3.) ** 0) * ((IB2 - 3.) ** 0)
    du[1] += 1.0 * c[1] * ((IB1 - 3.) ** 0) * ((IB2 - 3.) ** 0)
    du[0] += 2.0 * c[

In [26]:
F = UnsymMatrix('F')
A = SymMatrix('A')
X = F * A * F.T
print 'def push(F, A):'
ij = {0:(0,0),1:(1,1),2:(2,2),3:(0,1),4:(1,2),5:(0,2)}
print '    X = zeros(6)'
for I in range(6):
    i,j = ij[I]
    print '    X[{}] = {}'.format(I, X[i,j])
print '    return X / det(F)'

def push(F, A):
    X = zeros(6)
    X[0] = F[0]*(A[0]*F[0] + A[3]*F[1] + A[5]*F[2]) + F[1]*(A[1]*F[1] + A[3]*F[0] + A[4]*F[2]) + F[2]*(A[2]*F[2] + A[4]*F[1] + A[5]*F[0])
    X[1] = F[3]*(A[0]*F[3] + A[3]*F[4] + A[5]*F[5]) + F[4]*(A[1]*F[4] + A[3]*F[3] + A[4]*F[5]) + F[5]*(A[2]*F[5] + A[4]*F[4] + A[5]*F[3])
    X[2] = F[6]*(A[0]*F[6] + A[3]*F[7] + A[5]*F[8]) + F[7]*(A[1]*F[7] + A[3]*F[6] + A[4]*F[8]) + F[8]*(A[2]*F[8] + A[4]*F[7] + A[5]*F[6])
    X[3] = F[3]*(A[0]*F[0] + A[3]*F[1] + A[5]*F[2]) + F[4]*(A[1]*F[1] + A[3]*F[0] + A[4]*F[2]) + F[5]*(A[2]*F[2] + A[4]*F[1] + A[5]*F[0])
    X[4] = F[6]*(A[0]*F[3] + A[3]*F[4] + A[5]*F[5]) + F[7]*(A[1]*F[4] + A[3]*F[3] + A[4]*F[5]) + F[8]*(A[2]*F[5] + A[4]*F[4] + A[5]*F[3])
    X[5] = F[6]*(A[0]*F[0] + A[3]*F[1] + A[5]*F[2]) + F[7]*(A[1]*F[1] + A[3]*F[0] + A[4]*F[2]) + F[8]*(A[2]*F[2] + A[4]*F[1] + A[5]*F[0])
    return X / det(F)


In [27]:
F = UnsymMatrix('F')
A = SymMatrix('A')
Ai = A.inv()
Fi = F.inv()
ij = {0:(0,0),1:(1,1),2:(2,2),3:(0,1),4:(1,2),5:(0,2)}
print 'def inv(A):'
print '    if issym(A):'
print '        X = zeros(6)'
for I in ij:
    i,j = ij[I]
    print '        X[{}] = {}'.format(I, Ai[i,j])
print '    else:'
print '        X = np.linalg.inv(A.reshape(3,3))'
print '        X = X.reshape(9)'
print '    return X'

def inv(A):
    if issym(A):
        X = zeros(6)
        X[0] = -(A[0]*A[1] - A[3]**2)*(-A[3]*(A[4] - A[3]*A[5]/A[0])/(A[0]*(A[1] - A[3]**2/A[0])) + A[5]/A[0])*(A[3]*(A[4] - A[3]*A[5]/A[0])/(A[0]*(A[1] - A[3]**2/A[0])) - A[5]/A[0])/(A[0]*A[1]*A[2] - A[0]*A[4]**2 - A[1]*A[5]**2 - A[2]*A[3]**2 + 2*A[3]*A[4]*A[5]) + 1/A[0] + A[3]**2/(A[0]**2*(A[1] - A[3]**2/A[0]))
        X[1] = 1/(A[1] - A[3]**2/A[0]) + (A[4] - A[3]*A[5]/A[0])**2*(A[0]*A[1] - A[3]**2)/((A[1] - A[3]**2/A[0])**2*(A[0]*A[1]*A[2] - A[0]*A[4]**2 - A[1]*A[5]**2 - A[2]*A[3]**2 + 2*A[3]*A[4]*A[5]))
        X[2] = (A[0]*A[1] - A[3]**2)/(A[0]*A[1]*A[2] - A[0]*A[4]**2 - A[1]*A[5]**2 - A[2]*A[3]**2 + 2*A[3]*A[4]*A[5])
        X[3] = (A[4] - A[3]*A[5]/A[0])*(A[0]*A[1] - A[3]**2)*(-A[3]*(A[4] - A[3]*A[5]/A[0])/(A[0]*(A[1] - A[3]**2/A[0])) + A[5]/A[0])/((A[1] - A[3]**2/A[0])*(A[0]*A[1]*A[2] - A[0]*A[4]**2 - A[1]*A[5]**2 - A[2]*A[3]**2 + 2*A[3]*A[4]*A[5])) - A[3]/(A[0]*(A[1] - A[3]**2/A[0]))
        X[4] = -(A[4] - A[3]*A[5]/A[0])*(A[0

In [57]:
print 'def dyad(A, B):'
print '    if A.size == 6 and B.size == 6:'
print '        X = zeros((6,6))'
ij = {0:(0,0),1:(1,1),2:(2,2),3:(0,1),4:(1,2),5:(0,2)}
II = lambda m,n: dict([(v,k) for k,v in ij.items()])[m,n]
for I in range(6):
    for J in range(6):
        ij_ = II(*sorted(ij[I]))
        kl = II(*sorted(ij[J]))
        print '        X[{},{}] = A[{}] * B[{}]'.format(I,J,ij_,kl)
print '    elif A.size == 3 and B.size == 3:'
print '        X = zeros(6)'

for I in range(6):
    i,j = ij[I]
    print '        X[{}] = A[{}] * B[{}]'.format(I,i,j)
print '    else:\n        raise NotImplementedError'
print '    return X'

def dyad(A, B):
    if A.size == 6 and B.size == 6:
        X = zeros((6,6))
        X[0,0] = A[0] * B[0]
        X[0,1] = A[0] * B[1]
        X[0,2] = A[0] * B[2]
        X[0,3] = A[0] * B[3]
        X[0,4] = A[0] * B[4]
        X[0,5] = A[0] * B[5]
        X[1,0] = A[1] * B[0]
        X[1,1] = A[1] * B[1]
        X[1,2] = A[1] * B[2]
        X[1,3] = A[1] * B[3]
        X[1,4] = A[1] * B[4]
        X[1,5] = A[1] * B[5]
        X[2,0] = A[2] * B[0]
        X[2,1] = A[2] * B[1]
        X[2,2] = A[2] * B[2]
        X[2,3] = A[2] * B[3]
        X[2,4] = A[2] * B[4]
        X[2,5] = A[2] * B[5]
        X[3,0] = A[3] * B[0]
        X[3,1] = A[3] * B[1]
        X[3,2] = A[3] * B[2]
        X[3,3] = A[3] * B[3]
        X[3,4] = A[3] * B[4]
        X[3,5] = A[3] * B[5]
        X[4,0] = A[4] * B[0]
        X[4,1] = A[4] * B[1]
        X[4,2] = A[4] * B[2]
        X[4,3] = A[4] * B[3]
        X[4,4] = A[4] * B[4]
        X[4,5] = A[4] * B[5]
        X[5,0] = A[5] * B[0]
        X[5,1] = A[5] * 

In [45]:
from numpy import identity
d = identity(3, dtype=float)
ij = {0:(0,0),1:(1,1),2:(2,2),3:(0,1),4:(1,2),5:(0,2)}
II1 = []
II2 = []
II3 = []
II5 = []
for I in range(6):
    for J in range(6):
        i,j = ij[I]
        k,l = ij[J]
        II1.append('{0:.1f}'.format(d[i,j] * d[k,l]))
        II2.append('{0:.1f}'.format(d[i,k] * d[j,l]))
        II3.append('{0:.1f}'.format(d[i,l] * d[j,k]))
        II5.append('{0:.1f}'.format((d[i,k] * d[j,l] + d[i,l]*d[j,k])/2.))
                   
II1 = ', '.join(II1)
print '# II1: I[i,j] I[k,l]'
print 'II1 = array([{}]).reshape(6,6)'.format(II1)

II2 = ', '.join(II2)
print '# II2: I[i,k] I[j,l]'
print 'II2 = array([{}]).reshape(6,6)'.format(II2)

II3 = ', '.join(II3)
print '# II3: I[i,l] I[j,k]'
print 'II3 = array([{}]).reshape(6,6)'.format(II3)

print 'II4 = (II2 + II3) / 2'
II5 = ', '.join(II5)                   
print '# II5 = (I[i,k] I[j,l] + I[i,l] I[j,k]) / 2'
print 'II5 = array([{}]).reshape(6,6)'.format(II5)


# II1: I[i,j] I[k,l]
II1 = array([1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]).reshape(6,6)
# II2: I[i,k] I[j,l]
II2 = array([1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]).reshape(6,6)
# II3: I[i,l] I[j,k]
II3 = array([1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]).reshape(6,6)
II4 = (II2 + II3) / 2
# II5 = (I[i,k] I[j,l] + I[i,l] I[j,k]) / 2
II5 = array([1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5]).reshape(6,6)


In [58]:

print 'def symshuffle(A, B):'
print '    # Xijkl = .5 (Aik Bjl + Ail Bjk)'
print '    X = zeros((6,6))'
print '    if A.size == 6 and B.size == 6:'
ij = {0:(0,0),1:(1,1),2:(2,2),3:(0,1),4:(1,2),5:(0,2)}
II = lambda m,n: dict([(v,k) for k,v in ij.items()])[m,n]
for I in range(6):
    for J in range(6):
        i,j = ij[I]
        k,l = ij[J]
        ik = II(*sorted((i,k)))
        jl = II(*sorted((j,l)))
        il = II(*sorted((i,l)))
        jk = II(*sorted((j,k)))
        print '        X[{},{}] = (A[{}] * B[{}] + A[{}] * B[{}]) / 2.'.format(I,J,ik,jl,il,jk)
print '    else:'
print '        raise NotImplementedError'
print '    return X'



def symshuffle(A, B):
    # Xijkl = .5 (Aik Bjl + Ail Bjk)
    X = zeros((6,6))
    if A.size == 6 and B.size == 6:
        X[0,0] = (A[0] * B[0] + A[0] * B[0]) / 2.
        X[0,1] = (A[3] * B[3] + A[3] * B[3]) / 2.
        X[0,2] = (A[5] * B[5] + A[5] * B[5]) / 2.
        X[0,3] = (A[0] * B[3] + A[3] * B[0]) / 2.
        X[0,4] = (A[3] * B[5] + A[5] * B[3]) / 2.
        X[0,5] = (A[0] * B[5] + A[5] * B[0]) / 2.
        X[1,0] = (A[3] * B[3] + A[3] * B[3]) / 2.
        X[1,1] = (A[1] * B[1] + A[1] * B[1]) / 2.
        X[1,2] = (A[4] * B[4] + A[4] * B[4]) / 2.
        X[1,3] = (A[3] * B[1] + A[1] * B[3]) / 2.
        X[1,4] = (A[1] * B[4] + A[4] * B[1]) / 2.
        X[1,5] = (A[3] * B[4] + A[4] * B[3]) / 2.
        X[2,0] = (A[5] * B[5] + A[5] * B[5]) / 2.
        X[2,1] = (A[4] * B[4] + A[4] * B[4]) / 2.
        X[2,2] = (A[2] * B[2] + A[2] * B[2]) / 2.
        X[2,3] = (A[5] * B[4] + A[4] * B[5]) / 2.
        X[2,4] = (A[4] * B[2] + A[2] * B[4]) / 2.
        X[2,5] = (A[5] * B[2] + A[

## Push transformation

$$A_{ij} = F_{im}C_{mn}F_{jn} = L_{imjn}C_{mn}$$

where

$$ L_{imjn} = F_{im}F_{jn}$$

In [108]:
ij = {0:(0,0),1:(1,1),2:(2,2),3:(0,1),4:(1,2),5:(0,2)}
II = lambda m,n: dict([(v,k) for k,v in ij.items()])[m,n]
L = {}
for i in range(3):
    for j in range(3):
        I = II(*sorted((i,j)))
        row = []
        for m in range(3):
            for n in range(3):
                J = II(*sorted((m,n)))
                L.setdefault((I,J), []).append('F[{},{}] * F[{},{}]'.format(i,m,j,n))

for (k, v) in L.items():
    L[k] = ' + '.join(v[:2])

print 'def symleaf(F):'
print '    X = zeros((6,6))'
for i in range(6):
    for j in range(6):
        print '    X[{},{}] = {}'.format(i,j,L[i,j])
print '    return X'

def symleaf(F):
    X = zeros((6,6))
    X[0,0] = F[0,0] * F[0,0]
    X[0,1] = F[0,1] * F[0,1]
    X[0,2] = F[0,2] * F[0,2]
    X[0,3] = F[0,0] * F[0,1] + F[0,1] * F[0,0]
    X[0,4] = F[0,1] * F[0,2] + F[0,2] * F[0,1]
    X[0,5] = F[0,0] * F[0,2] + F[0,2] * F[0,0]
    X[1,0] = F[1,0] * F[1,0]
    X[1,1] = F[1,1] * F[1,1]
    X[1,2] = F[1,2] * F[1,2]
    X[1,3] = F[1,0] * F[1,1] + F[1,1] * F[1,0]
    X[1,4] = F[1,1] * F[1,2] + F[1,2] * F[1,1]
    X[1,5] = F[1,0] * F[1,2] + F[1,2] * F[1,0]
    X[2,0] = F[2,0] * F[2,0]
    X[2,1] = F[2,1] * F[2,1]
    X[2,2] = F[2,2] * F[2,2]
    X[2,3] = F[2,0] * F[2,1] + F[2,1] * F[2,0]
    X[2,4] = F[2,1] * F[2,2] + F[2,2] * F[2,1]
    X[2,5] = F[2,0] * F[2,2] + F[2,2] * F[2,0]
    X[3,0] = F[0,0] * F[1,0] + F[1,0] * F[0,0]
    X[3,1] = F[0,1] * F[1,1] + F[1,1] * F[0,1]
    X[3,2] = F[0,2] * F[1,2] + F[1,2] * F[0,2]
    X[3,3] = F[0,0] * F[1,1] + F[0,1] * F[1,0]
    X[3,4] = F[0,1] * F[1,2] + F[0,2] * F[1,1]
    X[3,5] = F[0,0] * F[1,2] + F[0,2] * F[1,